In [22]:
# setup
import numpy as np
import matplotlib.pyplot as plt

# https://www.wikiwand.com/en/Spirograph for mathematical formula
thetas = np.arange(-np.pi*100,np.pi*100,5)
rho =10
r = 80
R = 100
l = rho / r
k = r / R

X = []
Y = []
for i in range(0,len(thetas)):
    value = 2*R*((1-k)*np.cos(thetas[i])+l*k*np.cos((1-k)/k*thetas[i]))
    short = round(value,2) + 150
    withX = "X" + str(short)
    X.append(withX)

for i in range(0,len(thetas)):
    value = 2*R*((1-k)*np.sin(thetas[i])-l*k*np.sin((1-k)/k*thetas[i]))
    short = round(value,2) + 150
    withY = "Y" + str(short)
    Y.append(withY)

coordPairs = list(zip(X,Y))

# Adding g-code syntax
start = """M201 X500.00 Y500.00 Z100.00 E5000.00 ;Setup machine max acceleration
M203 X500.00 Y500.00 Z10.00 E50.00 ;Setup machine max feedrate
M204 P500.00 R1000.00 T500.00 ;Setup Print/Retract/Travel acceleration
M205 X8.00 Y8.00 Z0.40 E5.00 ;Setup Jerk
M220 S100 ;Reset Feedrate
M221 S100 ;Reset Flowrate

G28 ;Home
G21 ;Metric units

G1 Z0.2 F3000 ;Raise Z
"""

stop = """\nG91 ;Relative positioning
G1 Z5 F2400 ;Raise Z
G1 Z10 ;Raise Z more
G90 ;Absolute positionning

G1 X0 Y{machine_depth} ;Present print
M106 S0 ;Turn-off fan

M84 X Y E ;Disable all steppers but Z
"""

def g01(pair):
    return ("G01 " + pair[0] + " " + pair[1])

lines = list(map(g01, coordPairs))
list.insert(lines,1,lines[0]+" Z0")
lines[0] = lines[0] + " F2500"

#print(lines)

file = open("forfunspirograph.gcode", "w")
file.write(start + "\n")
for i in range(0,len(lines)):
    file.write(lines[i] + "\n")
file.write(stop)    
file.close()